# Assignment 1: Dino Fun World

### Assignment Description

You, in your role as a data explorer and visualizer, have been asked by the administrators of a small amusement park in your hometown to answer a few questions about their park operations. The dataset that they provided for you to perform the requested analysis includes the movement and communication data captured from the park attendees' apps during one weekend (Friday, Saturday, and Sunday).

The administrators would like you to answer four relatively simple questions about the park activities on the day in question. These questions all deal with park operations and can be answered using the data provided.
* **Question 1:** What is the most popular attraction to visit in the park?


* **Question 2:** What ride (note that not all attractions are rides) has the second longest average visit time?


* **Question 3:** Which Fast Food offering has the fewest visitors?


* **Question 4:** Compute the Skyline of number of visits and visit time for the park's ride, and report the rides that appear in the Skyline. (Note: Your answer should be three points, which can be given in any order.)


### Directions

The database provided by the park administration is formatted to be readable by any SQL database library. The course staff recommends the sqlite3 library. The database contains three tables, named 'checkin', 'attractions', and 'sequences'. The database file is named 'dinofunworld.db' .

The information contained in each of these tables is listed below:

`checkin`:
    - The check-in data for all visitors for the day in the park. The data includes two types of check-ins: inferred and actual checkins.
    - Fields: visitorID, timestamp, attraction, duration, type

`attraction`:
    - The attractions in the park by their corresponding AttractionID, Name, Region, Category, and type. Regions are from the VAST Challenge map such as Coaster Alley, Tundra Land, etc. Categories include Thrill rides, Kiddie Rides, etc. Type is broken into Outdoor Coaster, Other Ride, Carousel, etc.
    - Fields: AttractionID, Name, Region, Category, type
    
`sequences`:
    - The check-in sequences of visitors. These sequences list the position of each visitor to the park every five minutes. If the visitor has not entered the part yet, the sequence has a value of 0 for that time interval. If the visitor is in the park, the sequence lists the attraction they have most recently checked in to until they check in to a new one or leave the park.
    - Fields: visitorID, sequence
    
Using the provided data, answer the four questions that the administrators have asked.

### Submission Directions for Assignment Deliverables

This assignment will be auto-graded. We recommend that you submit this assignment. In order for your answers to be correctly registered in the system, you must place the code for your answers in the cell indicated for each question. In addition, you should submit the assignment with the output of the code in the cell's display area. The display area should contain only your answer to the question with no extraneous information, or else the answer may not be picked up correctly. 

Each cell that is going to be graded has a set of comment lines at the beginning of the cell. These lines are extremely important and must not be modified or removed. 

Please execute each cell in Jupyter Notebook before submitting.




### Question 1: 
What is the most popular attraction to visit in the park?

**Note:** Your output should be the name of the attraction.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import math
import datetime

In [3]:
import sqlite3
data = sqlite3.connect('/content/drive/MyDrive/DV/Assignment-1/dinofunworld.db')
db = data.cursor()

In [4]:
# your code here for Q1
db.execute("Select attraction, count(*) From checkin Group By attraction")
l = db.fetchall()
l1 = sorted(l,key = lambda x: x[1], reverse = True)
pop_attr_id = str(l1[0][0])
db.execute("SELECT Name FROM attraction WHERE AttractionID="+pop_attr_id+";")
pop_attraction = db.fetchall()[0][0]
print(pop_attraction)

Atmosfear


### Question 2: 

What ride (note that not all attractions are rides) has the  longest average visit time?

**Note:** Your output should be the name of the ride.


In [5]:
def transform_time(time):
    h, m, s = time.split(':')
    t_sec = int(datetime.timedelta(hours=int(h),minutes=int(m),seconds=int(s)).total_seconds())
    return t_sec

In [6]:
# your code here for Q2

#Getting the list of only rides
db.execute("SELECT AttractionID FROM attraction as a where a.category LIKE '%ride%';")
attraction_list = [i[0] for i in db.fetchall()]
max_id = 0
max_dur = 0
for a_id in attraction_list:
    db.execute("SELECT duration FROM checkin where attraction="+str(a_id)+";")
    duration_list = db.fetchall()
    total = 0
    for duration in duration_list:
        #Had to include try and except since few data is corrupted in the DB
        try:
            total += transform_time(duration[0])
        except:
            continue
    avg_dur = total /len(duration_list)
    if avg_dur > max_dur:
        max_dur = avg_dur
        max_id = a_id

db.execute("SELECT name FROM attraction WHERE AttractionID="+str(max_id)+";")
db.fetchall()[0][0]

'TerrorSaur'

### Question 3:

Which Fast Food offering in the park has the fewest visitors?

**Note:** Your output should be the name of the fast food offering.

In [7]:
# your code here for Q3
db.execute("SELECT AttractionID FROM attraction as a where a.category LIKE '%food%';")
attraction_list = [i[0] for i in db.fetchall()]

db.execute(" Select attraction, count(*) From checkin Group By attraction")
freq = db.fetchall() 

min_id = attraction_list[0]
min_val = freq[0][1]
for i in attraction_list[1:]:
  val = freq[i-1][1]
  if val < min_val:
    min_val = val
    min_id = i-1
db.execute("Select name from attraction where AttractionID="+str(min_id)+";")
db.fetchall()[0][0]

'Theresaur Food Stop'

### Question 4:

Compute the Skyline of number of visits and visit time for the park's ride and report the rides that appear in the Skyline. 

**Note:** Remember that in this case, higher visits is better and lower visit times are better. Your output should be formatted as a list of names of the rides in the Skyline. Your output should be three points, which can be given in any order(example output : ['Ride 1','Ride 2','Ride 3'] )


In [8]:
db.execute(" Select attraction, count(*) From checkin Group By attraction")
freq = db.fetchall() 

In [10]:
# your code here for Q4

#Higher Visits, Lower duration time
max_v_id  = 0
max_visits = 0
min_dur_id = 0
min_dur = math.inf
max_min_id = 0
max_min_visits = 0
max_min_dur = math.inf
for (a_id, f) in freq:
    db.execute("SELECT duration FROM checkin where attraction="+str(a_id)+";")
    duration_list = db.fetchall()
    total = 0
    n = 0
    for duration in duration_list:
        #Had to include try and except since few data is corrupted in the DB
        try:
            total += transform_time(duration[0])
            n+= 1
        except:
            if total == 0:
              total = math.inf
            continue
    avg_total = total /len(duration_list)
    if f > max_visits:
      max_v_id = a_id
      max_visits = f
    if avg_total < min_dur:
      min_dur_id = a_id
      min_dur = avg_total
    if f > max_min_visits and avg_total < max_min_dur:
      max_min_id = a_id
      max_min_visits = f
      max_min_dur = avg_total

In [11]:
db.execute("SELECT Name FROM attraction WHERE AttractionID="+str(max_v_id)+" OR AttractionID="+str(min_dur_id)+" OR AttractionID="+str(max_min_id)+";")
for i in db.fetchall():
  print(i[0])

Wrightiraptor Mountain
Atmosfear
Dykesadactyl Thrill
